In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('data_cleaned.csv')



In [113]:
print(f"Shape: {df.shape}")
df.head()

Shape: (404800, 27)


,age,gender,marital_status,education,monthly_salary,employment_type,years_of_employment,company_type,house_type,monthly_rent,...,existing_loans,current_emi_amount,credit_score,bank_balance,emergency_fund,emi_scenario,requested_amount,requested_tenure,emi_eligibility,max_monthly_emi
0,38.0,Female,Married,Professional,82600.0,Private,0.9,Mid-Size,Rented,20000.0,...,Yes,23700.0,660.0,303200.0,70200.0,Personal Loan Emi,850000.0,15,Not_Eligible,500.0
1,38.0,Female,Married,Graduate,21500.0,Private,7.0,Mnc,Family,0.0,...,Yes,4100.0,714.0,92500.0,26900.0,E-Commerce Shopping Emi,128000.0,19,Not_Eligible,700.0
2,38.0,Male,Married,Professional,86100.0,Private,5.8,Startup,Own,0.0,...,No,0.0,650.0,672100.0,324200.0,Education Emi,306000.0,16,Eligible,27775.0
3,58.0,Female,Married,High School,66800.0,Private,2.2,Mid-Size,Own,0.0,...,No,0.0,685.0,440900.0,178100.0,Vehicle Emi,304000.0,83,Eligible,16170.0
4,48.0,Female,Married,Professional,57300.0,Private,3.4,Mid-Size,Family,0.0,...,No,0.0,770.0,97300.0,28200.0,Home Appliances Emi,252000.0,7,Not_Eligible,500.0


Business Rationale: Financial ratios like debt-to-income, expense-to-income, and affordability score are classic indicators of loan repayment capacity used by banks and credit rating agencies.

Technically: These ratios help quantify an applicant’s disposable income, spending stress, and risk profile beyond raw pay/savings. They provide normalized, comparable values for applicants who may have widely varying incomes, expenses, or EMIs.

In [114]:
# Debt-to-Income Ratio
df['debt_to_income'] = df['current_emi_amount'] / (df['monthly_salary'] + 1)
# Expense-to-Income Ratio
expense_cols = ['monthly_rent','school_fees','college_fees','travel_expenses','groceries_utilities','other_monthly_expenses']
df['expense_to_income'] = df[expense_cols].sum(axis=1) / (df['monthly_salary'] + 1)
# Affordability Score
df['affordability_score'] = (df['monthly_salary'] - df[expense_cols].sum(axis=1) - df['current_emi_amount']) / (df['monthly_salary'] + 1)


- Business Rationale: Credit bureaus classify applicants into risk bands for fast loan eligibility filtering. Employment stability incorporates job tenure with age to proxy risk of job/income loss.
- Technically: Grouping credit scores enables non-linear interpretation in ML models. Employment stability feature recognizes that longer tenure at a given age may reduce risk.

In [115]:
def credit_risk(score):
    if score >= 750: return 'Excellent'
    elif score >= 700: return 'Good'
    elif score >= 650: return 'Fair'
    else: return 'Poor'

df['credit_risk_category'] = df['credit_score'].apply(credit_risk)
df['employment_stability'] = df['years_of_employment'] / (df['age'] + 1)


Business Rationale: Many credit models use multiplicative or interactive features because relationships are rarely linear (e.g., high salary alone does not mean low risk – combine with credit score).

Technically: Interaction features allow ML algorithms to capture complex, non-linear risk signals and applicant variants. Loan-to-income ratio normalizes requested amount by earning capacity.

In [116]:
df['income_credit_interaction'] = df['monthly_salary'] * df['credit_score']
df['loan_to_income'] = df['requested_amount'] / ((df['monthly_salary'] + 1) * 12)


In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404800 entries, 0 to 404799
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   age                        404800 non-null  float64
 1   gender                     404800 non-null  object 
 2   marital_status             404800 non-null  object 
 3   education                  404800 non-null  object 
 4   monthly_salary             404800 non-null  float64
 5   employment_type            404800 non-null  object 
 6   years_of_employment        404800 non-null  float64
 7   company_type               404800 non-null  object 
 8   house_type                 404800 non-null  object 
 9   monthly_rent               404800 non-null  float64
 10  family_size                404800 non-null  int64  
 11  dependents                 404800 non-null  int64  
 12  school_fees                404800 non-null  float64
 13  college_fees               40

In [118]:
df.select_dtypes(include=['float64', 'int64']).columns

Index(['age', 'monthly_salary', 'years_of_employment', 'monthly_rent',
       'family_size', 'dependents', 'school_fees', 'college_fees',
       'travel_expenses', 'groceries_utilities', 'other_monthly_expenses',
       'current_emi_amount', 'credit_score', 'bank_balance', 'emergency_fund',
       'requested_amount', 'requested_tenure', 'max_monthly_emi',
       'debt_to_income', 'expense_to_income', 'affordability_score',
       'employment_stability', 'income_credit_interaction', 'loan_to_income'],
      dtype='object')

 Business Rationale: Outliers can distort ML models, making them focus disproportionately on rare cases. Capping at 1st/99th percentiles balances data preservation and extreme value mitigation.

 Technically: IQR is effective for outlier detection, but capping at percentiles instead of removing preserves record count (sample size), prevents model bias, and ensures stable downstream scaling.

In [119]:
# List of continuous features to cap
iqr_cols = ['monthly_salary', 'years_of_employment', 'monthly_rent', 'college_fees',
    'travel_expenses', 'groceries_utilities', 'other_monthly_expenses',
    'current_emi_amount', 'credit_score', 'bank_balance', 'emergency_fund',
    'requested_amount', 'requested_tenure', 'max_monthly_emi']

for col in iqr_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    # Cap values outside IQR range
    df[col] = np.where(df[col] < lower, lower, np.where(df[col] > upper, upper, df[col]))


In [120]:
from sklearn.preprocessing import StandardScaler

scale_cols = ['monthly_salary', 'years_of_employment', 'monthly_rent', 'college_fees',
    'travel_expenses', 'groceries_utilities', 'other_monthly_expenses',
    'current_emi_amount', 'credit_score', 'bank_balance', 'emergency_fund',
    'requested_amount', 'requested_tenure', 'max_monthly_emi']

scaler = StandardScaler()
df[scale_cols] = scaler.fit_transform(df[scale_cols])


Based on EDA summary, the following columns should be capped:

monthly_salary

years_of_employment

monthly_rent

college_fees

travel_expenses

groceries_utilities

other_monthly_expenses

current_emi_amount

credit_score

bank_balance

emergency_fund

requested_amount

requested_tenure

max_monthly_emi

You have also listed:

age (0 outliers, safe to skip)

family_size and dependents (0 outliers, not needed for capping)

school_fees (0 outliers, not needed for capping)

Best Practice for Outlier Handling
Include all listed columns with actual outliers in your IQR-based capping code.

Do not cap columns with 0 outliers: This avoids unnecessary transformation and preserves genuine variance.

In [121]:
df.select_dtypes(include=["object"]).columns

Index(['gender', 'marital_status', 'education', 'employment_type',
       'company_type', 'house_type', 'existing_loans', 'emi_scenario',
       'emi_eligibility', 'credit_risk_category'],
      dtype='object')

Business Rationale: ML algorithms require numeric input. Label encoding is used for binary/ordinal features, and one-hot encoding for nominal multi-category variables.

Technically: Label encoding preserves order (if any) and one-hot encoding prevents models from assigning artificial order to nominal data; both increase model flexibility and avoid misleading value interpretations.

In [122]:
from sklearn.preprocessing import LabelEncoder

# Label encode binary categoricals
for col in ['gender','marital_status','existing_loans']:
    df[col+'_enc'] = LabelEncoder().fit_transform(df[col])

# One-hot encode larger categoricals
df = pd.get_dummies(df, columns=['education','employment_type','company_type','house_type','emi_scenario','credit_risk_category'], drop_first=True)


In [123]:
# Example: Cap features at 1st and 99th percentiles
'''for col in ['monthly_salary','years_of_employment','credit_score','bank_balance','requested_amount','max_monthly_emi']:
    lower = df[col].quantile(0.01)
    upper = df[col].quantile(0.99)
    df[col] = np.where(df[col]<lower,lower,np.where(df[col]>upper,upper,df[col]))'''


"for col in ['monthly_salary','years_of_employment','credit_score','bank_balance','requested_amount','max_monthly_emi']:\n    lower = df[col].quantile(0.01)\n    upper = df[col].quantile(0.99)\n    df[col] = np.where(df[col]<lower,lower,np.where(df[col]>upper,upper,df[col]))"

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404800 entries, 0 to 404799
Data columns (total 49 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   age                               404800 non-null  float64
 1   gender                            404800 non-null  object 
 2   marital_status                    404800 non-null  object 
 3   monthly_salary                    404800 non-null  float64
 4   years_of_employment               404800 non-null  float64
 5   monthly_rent                      404800 non-null  float64
 6   family_size                       404800 non-null  int64  
 7   dependents                        404800 non-null  int64  
 8   school_fees                       404800 non-null  float64
 9   college_fees                      404800 non-null  float64
 10  travel_expenses                   404800 non-null  float64
 11  groceries_utilities               404800 non-null  f

In [125]:
print(df.shape)
df.to_csv('data_feature_engineered.csv', index=False)
print("✓ Feature engineering complete and saved.")


(404800, 49)
✓ Feature engineering complete and saved.


All features needing outlier treatment were capped appropriately (using IQR).

Categorical, ratio, and interaction features are encoded and engineered.

No unhandled missing values or data leakage columns remain.

Dataset is clean and aligned with best practices for financial ML modeling.